In [ ]:
## Install Dependencies
!pip install dspy transformers torch accelerate

!pip install --upgrade transformers

## Import Required Libraries
import dspy
import torch
from transformers import AutoModelForVision2Seq, Blip2Processor

# Check for GPU & Enable Offloading
device = "cuda" if torch.cuda.is_available() else "cpu"
offload_buffers = True  # Reduce memory usage for large models

# Load Proxy-Lite-3B model
model_name = "convergence-ai/proxy-lite-3b"

# Load processor (handles text input)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b", trust_remote_code=True)

# Load the model with offloading enabled
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_buffers=offload_buffers,
    trust_remote_code=True
)

## Define Custom DSPy Model Wrapper
class CustomProxyLiteLLM(dspy.LM):
    def __init__(self, model, processor):
        super().__init__(model=model, provider="transformers")
        self.model = model
        self.processor = processor

    def complete(self, prompt, **kwargs):
        # Ensure correct input format
        inputs = self.processor(prompt, return_tensors="pt").to("cuda")

        # Generate response
        with torch.no_grad():
            output = self.model.generate(**inputs, max_length=512, do_sample=True, temperature=0.7)

        # Decode output
        return self.processor.tokenizer.decode(output[0], skip_special_tokens=True)

## Configure DSPy to Use Proxy-Lite-3B
llm = CustomProxyLiteLLM(model, processor)
dspy.settings.configure(lm=llm)

## Define a Prompt Optimization Task
class MyPromptOptimizationTask(dspy.Signature):
    query = dspy.InputField()
    response = dspy.OutputField(desc="An optimized response to the query")

class PromptOptimizer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.Predict(MyPromptOptimizationTask)

    def forward(self, query):
        return self.generator(query=query)

## Test the Model with a Sample Query
optimizer = PromptOptimizer()
output = optimizer("Explain black holes in simple terms.")
print(output.response)


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.4/260.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

TypeError: argument of type 'Qwen2_5_VLForConditionalGeneration' is not iterable